In [31]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import copy

In [32]:
dataset = pd.read_csv('apparel-trainval.csv')
# dataset

In [33]:
X = dataset.drop('label', axis = 1)
Y = dataset['label']

In [34]:
X = (X - np.mean(X))/np.std(X)

In [35]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)

In [36]:
X_train = X_train.iloc[:]
# X_copy = copy.deepcopy(X_train)
# Y_copy = copy.deepcopy(Y_train)
# Y_train
Y_train=pd.concat([Y_train,pd.get_dummies(Y_train,prefix='label')],axis=1).drop(['label'],axis=1)
Y_train = Y_train.iloc[:]

In [37]:
class Neural_network(object):
    
    def __init__(self):
        self.input_layer_size = 784
        self.output_layer_size = 10
        self.hidden_layer_size = 70
        self.hidden_layers = 1
        self.weights = {}
        self.dweights = {}
        self.Z = {}
        self.a = {}
        self.Y_hat = np.array([])
        self.delta = {}
        self.epoch = 300
        self.ita = 0.001
        
    def initialization(self):
        
        self.weights[1] = np.random.randn(self.input_layer_size,self.hidden_layer_size)*0.01
        self.weights[self.hidden_layers+1] = np.random.randn(self.hidden_layer_size,self.output_layer_size)*0.01
        
        for i in np.arange(2,self.hidden_layers+1):
            self.weights[i] = np.random.randn(self.hidden_layer_size,self.hidden_layer_size)*0.01
            
    
    def softmax(self,a):
        tsum = np.sum(a,axis = 1,keepdims = True)
        soft_a = np.divide(a,tsum)
        return soft_a
    
#     def y_prep(self,Y,k):
#          # Onehot
#         y = np.zeros((len(Y),self.output_layer_size))
#         for i in range(y.shape[0]):
#             y[i][Y[i]] = 1
#         return y
    
    
    def error(self,y):
        return sum(0.5*np.sum((y-self.Y_hat)*(y-self.Y_hat),axis = 1))
    
    def derivative_sigmoid(self,layer):
        derivative = layer*(1-layer)
        return derivative
    
   
    def deltas(self,y):
        w = self.a[self.hidden_layers+2]
        self.delta[self.hidden_layers+2] = -(y - self.Y_hat)*self.derivative_sigmoid(w)
        self.dweights[self.hidden_layers+1] = np.dot(self.a[self.hidden_layers+1].T , (self.delta[self.hidden_layers+2]))
        
        for i in np.arange(self.hidden_layers+1,1,-1):
            self.delta[i] = ( np.dot(self.delta[i+1], self.weights[i].T))*self.derivative_sigmoid(self.a[i])
            self.dweights[i-1] = np.dot(self.a[i-1].T , (self.delta[i]))  
    
    def sigmoid(self,x):
        x = -x
        return 1 / (1 + np.exp(x))
    
    
    
    def forward_propagation(self, X):
        self.a[1]=np.array(X)
        

        for i in range(1,self.hidden_layers+2):
            self.Z[i+1] = np.dot(self.a[i],self.weights[i])
            self.a[i+1] = self.sigmoid(self.Z[i+1])
        self.Y_hat = self.softmax(self.a[self.hidden_layers+2])
        
   
    def gradient_descent(self,X_train,Y_train):
        k = 0
        while k<=47999:
#             X=X_train.iloc[k:k+1000]
#             Y=Y_train.iloc[k:k+1000]
            X = X_train[k:k+1000]
            y = Y_train[k:k+1000]
#             print(y)
            for i in range(self.epoch):
                self.forward_propagation(X)
#                 y = self.y_prep(Y,k)
#                 print("Error : ",self.error(y))
                self.deltas(y)
                for j in range(1,self.hidden_layers+2):
                    self.weights[j] = self.weights[j] - self.ita*self.dweights[j]
            print("Final Error : ",self.error(y))
            k = k + 1000
            print(k/1000)
        
        
    def predict(self,X_test):
        self.forward_propagation(X_test)
        predicted = np.argmax(self.Y_hat,axis = 1)
        return predicted
    
    def accuracy(self,Y):
        predicted = np.argmax(self.Y_hat,axis = 1)
        print(predicted)
        print("Accuracy score : ",100*(accuracy_score(predicted,np.array(Y))))
            

In [38]:
k = Neural_network()
k.initialization()
#k.forward_propagation(X_train)
k.gradient_descent(X_train,Y_train)

Final Error :  129.3330029610473
1.0
Final Error :  69.61663378471718
2.0
Final Error :  67.50599074676474
3.0
Final Error :  46.79425149337116
4.0
Final Error :  50.0284406839285
5.0
Final Error :  53.06084280568302
6.0
Final Error :  53.945191627719595
7.0
Final Error :  57.18298607077054
8.0
Final Error :  46.18202941352885
9.0
Final Error :  50.32846785370816
10.0
Final Error :  43.997123288885035
11.0
Final Error :  53.527367190460666
12.0
Final Error :  51.16176149959548
13.0
Final Error :  50.57419034531504
14.0
Final Error :  51.46881908320289
15.0
Final Error :  46.38260133396649
16.0
Final Error :  58.99916023792801
17.0
Final Error :  51.88741221901272
18.0
Final Error :  54.80607864986095
19.0
Final Error :  59.74411376515447
20.0
Final Error :  63.32083490082194
21.0
Final Error :  66.36448282667472
22.0
Final Error :  59.02207401384976
23.0
Final Error :  58.82178382223992
24.0
Final Error :  63.08198219998459
25.0
Final Error :  53.176163015059764
26.0
Final Error :  60.

In [39]:
# X_test = X_test[0:1000]
# Y_test = Y_test[0:1000]

k.predict(X_test)
k.accuracy(Y_test)

[3 6 7 ... 8 2 9]
Accuracy score :  85.24166666666667


In [40]:
X_test = pd.read_csv('apparel-test.csv')
X_test = (X_test - np.mean(X_test))/np.std(X_test)
ans = k.predict(X_test)
ans_df = pd.DataFrame(ans)
ans_df.to_csv('predict.csv',index=False)